In [1]:
import pandas as pd


tasks = pd.read_parquet('../data/only_valid_pids/GANSLIDER_TASKS_TRANSFORMED.parquet')
drop_duplicates = tasks[['taskid', 'completed']].drop_duplicates()
drop_duplicates = tasks[~tasks.taskid.duplicated()]
tasks = tasks[tasks.taskid.isin(drop_duplicates.taskid)]
tasks

conf = pd.read_parquet('../data/only_valid_pids/GANSLIDER_CONFIGS_FLAT.parquet')
conf = conf[['taskid', 'slider_type', 'num_sliders', 'slider_id', 'target_value', 'seed']].drop_duplicates()

merged = tasks.merge(conf, on=['taskid', 'slider_id'])
merged

,taskid,completed,end_utc_timestamp_ms,start_utc_timestamp_ms,duration_sec,slider_id,edit_value,__index_level_0__,slider_type,num_sliders,target_value,seed
0,0024922e-53cb-43ce-9e3d-746bac2ad866,DONE,1.626976e+12,1.626976e+12,298.637,0,2.9,0,FILMSTRIP,10,2.0,7402
1,0024922e-53cb-43ce-9e3d-746bac2ad866,DONE,1.626976e+12,1.626976e+12,298.637,1,0.3,0,FILMSTRIP,10,4.4,7402
2,0024922e-53cb-43ce-9e3d-746bac2ad866,DONE,1.626976e+12,1.626976e+12,298.637,2,0.6,0,FILMSTRIP,10,-0.6,7402
3,0024922e-53cb-43ce-9e3d-746bac2ad866,DONE,1.626976e+12,1.626976e+12,298.637,3,1.8,0,FILMSTRIP,10,1.6,7402
4,0024922e-53cb-43ce-9e3d-746bac2ad866,DONE,1.626976e+12,1.626976e+12,298.637,4,-4.0,0,FILMSTRIP,10,-1.6,7402
...,...,...,...,...,...,...,...,...,...,...,...,...
9677,ffb40708-446e-4faa-a5c3-2fde69e7fc1f,DONE,1.628691e+12,1.628691e+12,200.798,3,-2.0,2145,FILMSTRIP,8,-4.6,7333
9678,ffb40708-446e-4faa-a5c3-2fde69e7fc1f,DONE,1.628691e+12,1.628691e+12,200.798,4,-4.0,2145,FILMSTRIP,8,-1.5,7333
9679,ffb40708-446e-4faa-a5c3-2fde69e7fc1f,DONE,1.628691e+12,1.628691e+12,200.798,5,-1.9,2145,FILMSTRIP,8,-1.0,7333
9680,ffb40708-446e-4faa-a5c3-2fde69e7fc1f,DONE,1.628691e+12,1.628691e+12,200.798,6,0.2,2145,FILMSTRIP,8,2.7,7333


In [2]:
import sys
from pathlib import Path
sys.path.append(str(Path('../backend').resolve()))

from src.image_generator import PCAStyleGan

StyleGAN2: Optimized CUDA op FusedLeakyReLU not available, using native PyTorch fallback.
StyleGAN2: Optimized CUDA op UpFirDn2d not available, using native PyTorch fallback.


In [3]:
image_generator = PCAStyleGan()

In [9]:
import numpy as np
import matplotlib.pyplot as plt

# Sample five tasks for each slider type, number of slider and completed status
samples = merged[['taskid', 'slider_type', 'num_sliders', 'completed']].drop_duplicates().groupby(['slider_type', 'num_sliders', 'completed'])['taskid'].apply(lambda arr: np.random.choice(list(arr), min(5, arr.shape[0]), replace=False))

for idx, rv in samples.items():
	fig, axes = plt.subplots(ncols=10, figsize=(20, 3))
	i = 0

	for taskid in rv:
		sample_task = merged.loc[merged.taskid == taskid, ['slider_id', 'edit_value', 'target_value', 'seed']]
		seed = sample_task.iloc[0]['seed']
		
		user_edits = sample_task[['slider_id', 'edit_value']].rename(columns={'slider_id': 'id', 'edit_value': 'value'})
		user_edits = user_edits.to_dict(orient='records')

		target_edits = sample_task[['slider_id', 'target_value']].rename(columns={'slider_id': 'id', 'target_value': 'value'})
		target_edits = target_edits.to_dict(orient='records')
		
		user_img = image_generator.generate_image(edits=user_edits, seed=seed, size=256)
		targ_img = image_generator.generate_image(edits=target_edits, seed=seed, size=256)

		axes[i].imshow(user_img)
		axes[i+1].imshow(targ_img)
		
		plt.title(idx)

		i = i+2

	plt.savefig(f'../images/sample_reconstruction_{idx[0]}_{idx[1]}_{idx[2]}.png')
	plt.close()